In [1]:
%pip install --quiet ibm-watsonx-ai python-dotenv tqdm scikit-learn



Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Imports & auth setup
import os
import json
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from dotenv import load_dotenv
from getpass import getpass

# WatsonX 
from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters


In [ ]:

# .env for key
load_dotenv()
api_key    = os.getenv("WATSONX_API_KEY") or getpass("IBM Cloud API key: ")
url        = os.getenv("WATSONX_URL")
project_id = os.getenv("WATSONX_PROJECT_ID")

# Build client
creds  = Credentials(url=url, api_key=api_key)
client = APIClient(credentials=creds, project_id=project_id)





python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 8


In [ ]:
# Instantiate LLM 
params = TextGenParameters(
    temperature=0.0,
    max_new_tokens=64,
    stop_sequences=["}"]
)
model = ModelInference(
    api_client=client,
    model_id="ibm/granite-13b-instruct-v2",
    params=params
)

In [ ]:

#Load & clean data
# %%
train_df = pd.read_csv("data/kaggle/train.csv")
test_df  = pd.read_csv("data/kaggle/test.csv")
tl_df    = pd.read_csv("data/kaggle/test_labels.csv")

label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"https?://\S+", " ", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

train_df['comment_text'] = train_df['comment_text'].apply(clean_text)
test_df['comment_text']  = test_df['comment_text'].apply(clean_text)

In [ ]:
# evaluation subset
# sample for prompt tuning
sample_df = train_df.sample(n=5000, random_state=42).reset_index(drop=True)


In [ ]:
#Filter to scored subset
scored = tl_df[label_cols].ne(-1).all(axis=1)
scored_ids = tl_df.loc[scored, 'id']
eval_df    = test_df[test_df['id'].isin(scored_ids)].reset_index(drop=True)
y_true     = tl_df.set_index('id').loc[eval_df['id'], label_cols].values.astype(int)

In [ ]:
#JSON-schema prompt
SYSTEM_PROMPT = """
You are a toxicity classifier.  Given the comment under TEXT, return ONLY a JSON object
with these six keys (true or false values):
  {keys}

Example:
TEXT: "You are an idiot."
{{"toxic": true, "severe_toxic": false, "obscene": false, 
  "threat": false, "insult": true, "identity_hate": false}}

Now classify this comment:
TEXT: {text}
"""


# pre-format the keys snippet
KEYS_JSON = json.dumps({k: False for k in label_cols})

In [ ]:
# Loop & call the LLM
preds = []
for txt in tqdm(eval_df['comment_text'], desc="LLM classify"):
    prompt = SYSTEM_PROMPT.format(keys=KEYS_JSON, text=txt)
    resp   = model.generate(prompt)
    raw    = resp['results'][0]['generated_text'].strip()
    
    # extract the first {...} block
    if '{' in raw and '}' in raw:
        raw = raw[raw.find('{'): raw.rfind('}')+1]
    try:
        d = json.loads(raw)
    except json.JSONDecodeError:
        d = {k: False for k in label_cols}
    preds.append([int(bool(d.get(k, False))) for k in label_cols])

y_pred = np.array(preds)

# for full eval on test set
scored_mask = tl_df[label_cols].ne(-1).all(axis=1)
scored_ids  = tl_df.loc[scored_mask, 'id']
eval_df     = test_df[test_df['id'].isin(scored_ids)].reset_index(drop=True)
y_true      = tl_df.set_index('id').loc[eval_df['id'], label_cols].values.astype(int)

LLM classify:   0%|          | 0/63978 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Evaluate
from sklearn.metrics import classification_report, roc_auc_score

print("ROC-AUC per label:")
for i, lbl in enumerate(label_cols):
    print(f"  {lbl:15s}: {roc_auc_score(y_true[:,i], y_pred[:,i]):.4f}")

print("\nClassification report:")
print(classification_report(y_true, y_pred, target_names=label_cols, zero_division=0))